# Exercise 7

In [113]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=10000, noise=0.4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [114]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier


params = {'max_leaf_nodes': list(range(2, 50)), 'min_samples_split': [2, 3, 4]}
tree_gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1,cv=3,verbose=2)

In [115]:
tree_gs.fit(X_train,y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  44 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:    2.2s finished


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]},
             verbose=2)

In [116]:
tree_gs.best_params_

{'max_leaf_nodes': 36, 'min_samples_split': 2}

In [117]:
y_pred = tree_gs.best_estimator_.predict(X_test)

In [118]:
from sklearn.metrics import accuracy_score

accuracy_score(y_pred, y_test)

0.856

# Exercise 8

In [119]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)

In [120]:
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

In [121]:
from sklearn.base import clone

forest = [clone(tree_gs.best_estimator_) for j in range(n_trees)]

In [122]:
accuracy_scores = []

In [123]:
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train,y_mini_train)
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test,y_pred))

In [124]:
import numpy as np

np.mean(accuracy_scores)

0.7909155

In [125]:
Y_pred = np.empty([n_trees, len(X_test)])

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)           

In [126]:
from scipy.stats import mode

y_pred_mode, n_votes = mode(Y_pred, axis=0)

In [127]:
accuracy_score(y_test, y_pred_mode.reshape([-1]))

0.868